# Simple Chatbot

In [32]:
from dotenv import load_dotenv

load_dotenv()

True

In [28]:
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, START, END
from typing import TypedDict

# Define state (no memory, only current input/output)
class State(TypedDict):
    input: str
    output: str

llm = ChatOpenAI(model="gpt-4o-mini")

# Simple agent node 
def agent_node(state: State):
    response = llm.invoke(state["input"])
    return {"output": response.content}

# Build graph
graph = StateGraph(State)
graph.add_node("agent", agent_node)
graph.add_edge(START, "agent")
graph.add_edge("agent", END)

app_without_memory = graph.compile()

# Run 
print("---- Without Memory ----")
print(app_without_memory.invoke({"input": "What is  name is OpenAI"}))
print(app_without_memory.invoke({"input": "Where is it located?"}))


---- Without Memory ----
{'input': 'What is  OpenAI', 'output': 'OpenAI is an artificial intelligence research organization that aims to ensure that artificial general intelligence (AGI) benefits all of humanity. Founded in December 2015, OpenAI conducts research in various areas of AI, including natural language processing, machine learning, and robotics. It is known for developing advanced AI models, such as the GPT (Generative Pre-trained Transformer) series, which includes language models capable of understanding and generating human-like text.\n\nOpenAI also emphasizes safety and ethical considerations in AI development, advocating for responsible usage of AI technology. The organization operates with a unique blend of nonprofit and for-profit structures, allowing it to pursue its mission while attracting funding to support its ambitious research and development goals.'}
{'input': 'Where is it located?', 'output': 'Could you please provide more context or specify what you are aski

# Simple Chatbot with Memory 

In [30]:
from typing import TypedDict, Annotated
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

# State now keeps structured messages
class State(TypedDict):
    messages: Annotated[list, add_messages]

llm = ChatOpenAI(model="gpt-4o-mini")

# Node 
def agent_node(state: State):
    response = llm.invoke(state["messages"])
    return {"messages": [response]}  

# Build graph
graph = StateGraph(State)
graph.add_node("agent", agent_node)
graph.add_edge(START, "agent")
graph.add_edge("agent", END)
app_with_memory = graph.compile()

# ---- Demo ----
print("---- With Memory ----")

# First turn
state = {"messages": [HumanMessage(content="What is OpenAI?")]}
state = app_with_memory.invoke(state)
print("Assistant:", state["messages"][-1].content)

# Second turn (follow-up)
state["messages"].append(HumanMessage(content="Where is it located?"))
state = app_with_memory.invoke(state)
print("Assistant:", state["messages"][-1].content)


---- With Memory ----
Assistant: OpenAI is an artificial intelligence research organization that aims to develop and promote friendly AI for the benefit of humanity. Founded in December 2015, OpenAI focuses on creating advanced AI technologies while ensuring that these technologies are safe and beneficial. The organization conducts research in various areas of AI, including machine learning, reinforcement learning, natural language processing, and robotics.

OpenAI is known for several prominent projects, including the development of the GPT (Generative Pre-trained Transformer) models, which are capable of understanding and generating human-like text. The organization also emphasizes the importance of AI safety and responsible AI deployment, advocating for policies that ensure AI is aligned with human values and can be controlled.

In addition to research, OpenAI engages in partnerships, open-source initiatives, and collaborations to share knowledge and promote the responsible use of A